In [7]:
import baostock as bs
import pandas as pd
import math
import numpy as np
from sklearn.preprocessing import MinMaxScaler

#### 登陆系统 ####
lg = bs.login()
# 显示登陆返回信息
# print('login respond error_code:'+lg.error_code)
# print('login respond  error_msg:'+lg.error_msg)

#### 获取沪深A股历史K线数据 ####
# 详细指标参数，参见“历史行情指标参数”章节；“分钟线”参数与“日线”参数不同。“分钟线”不包含指数。
# 分钟线指标：date,time,code,open,high,low,close,volume,amount,adjustflag
# 周月线指标：date,code,open,high,low,close,volume,amount,adjustflag,turn,pctChg
rs = bs.query_history_k_data_plus("sh.600000",
    "time,open,high,low,close",
    start_date='2005-01-01', end_date='2023-12-31',
    frequency="5", adjustflag="2")

#### 打印结果集 ####
data_list = []
while (rs.error_code == '0') & rs.next():
    # 获取一条记录，将记录合并在一起
    data_list.append(rs.get_row_data())
df = pd.DataFrame(data_list, columns=rs.fields)

for i in range(0,len(df.time)):
    # Convert the string into a format pandas can handle
    time_str = df.loc[i, 'time']
    time_str = time_str[:4] + "-" + time_str[4:6] + "-" + time_str[6:8] + " " + time_str[8:10] + ":" + time_str[10:12] + ":" + time_str[12:14] + "." + time_str[14:]
    
    # Convert the string into a datetime object
    df.loc[i, 'time'] = pd.to_datetime(time_str)

df.index=pd.to_datetime(df.time)
#### 结果集输出到csv文件 ####   
print(df)

#### 登出系统 ####
bs.logout()

# create the dataset only containing close prices
data = pd.DataFrame(pd.to_numeric(df["close"]))
dataset = np.reshape(data.values, (df.shape[0], 1))

# # normalise the dataset
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

# define a function to get training and test data
def split_data(dataset, train_time, predict_time):
    x = []
    y = []
    for i in range(train_time, len(dataset)-predict_time+1):
        x.append(dataset[i-train_time : i, 0])
        y.append(dataset[i+predict_time-1, 0])
    return x, y

# x => data from previous 25 mins; y => data in the next 5 mins
def reshape_data(train_data, test_data, fmins):
    x_train, y_train = split_data(train_data, fmins, 1)
    x_test, y_test = split_data(test_data, fmins, 1)
    # convert data into numpy arrays
    x_train, y_train = np.array(x_train), np.array(y_train)
    x_test, y_test = np.array(x_test), np.array(y_test)
    # reshape the data for neural network training
    x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
    x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))
    return x_train, y_train, x_test, y_test

# create the scaled training data set
training_data_len = math.ceil(len(dataset) * 0.9087)
train_data = scaled_data[0:training_data_len, :]
#print(data[:train_data.shape[0]].tail())

# create the scaled test data set
test_data = scaled_data[training_data_len-5: , :]

# use 5 consecutive trading minutes as the unit step size sliding through the stock price data
x_train_5, y_train_5, x_test_5, y_test_5 = reshape_data(train_data, test_data, 5)
print("when sequence length is 5, data shape:", x_train_5.shape, y_train_5.shape, x_test_5.shape, y_test_5.shape)

login success!
                                    time          open          high  \
time                                                                   
2005-01-04 09:35:00  2005-01-04 09:35:00  0.9099128000  0.9099128000   
2005-01-04 09:40:00  2005-01-04 09:40:00  0.9007876000  0.9020912000   
2005-01-04 09:45:00  2005-01-04 09:45:00  0.8981804000  0.9007876000   
2005-01-04 09:50:00  2005-01-04 09:50:00  0.8981804000  0.8994840000   
2005-01-04 09:55:00  2005-01-04 09:55:00  0.8968768000  0.8981804000   
...                                  ...           ...           ...   
2023-12-29 14:40:00  2023-12-29 14:40:00  6.6200000000  6.6300000000   
2023-12-29 14:45:00  2023-12-29 14:45:00  6.6200000000  6.6300000000   
2023-12-29 14:50:00  2023-12-29 14:50:00  6.6200000000  6.6300000000   
2023-12-29 14:55:00  2023-12-29 14:55:00  6.6200000000  6.6300000000   
2023-12-29 15:00:00  2023-12-29 15:00:00  6.6200000000  6.6300000000   

                              low         close 

In [8]:
# download CSI dataset
lg = bs.login()

fields= "date,open,high,low,close"
rs = bs.query_history_k_data("sh.000001", fields, start_date="2022-01-01", end_date="2023-12-31", frequency="d", adjustflag="2")
data_list = []
while (rs.error_code == "0") & rs.next():
    data_list.append(rs.get_row_data())
df = pd.DataFrame(data_list, columns=rs.fields)

df.index=pd.to_datetime(df.date)

bs.logout()

print(df)

# # create the dataset only containing close prices
# data = pd.DataFrame(pd.to_numeric(df["close"]))
# dataset = np.reshape(data.values, (df.shape[0], 1))
# 
# # # normalise the dataset
# scaler = MinMaxScaler(feature_range=(0,1))
# scaled_data = scaler.fit_transform(dataset)
# 
# # define a function to get training and test data
# def split_data(dataset, train_day, predict_day):
#     x = []
#     y = []
#     for i in range(train_day, len(dataset)-predict_day+1):
#         x.append(dataset[i-train_day : i, 0])
#         y.append(dataset[i+predict_day-1, 0])
#     return x, y
# 
# # x => data from previous days; y => data in the next day
# def reshape_data(train_data, test_data, days):
#     x_train, y_train = split_data(train_data, days, 1)
#     x_test, y_test = split_data(test_data, days, 1)
#     # convert data into numpy arrays
#     x_train, y_train = np.array(x_train), np.array(y_train)
#     x_test, y_test = np.array(x_test), np.array(y_test)
#     # reshape the data for neural network training
#     x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
#     x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))
#     return x_train, y_train, x_test, y_test
# 
# # create the scaled training data set
# training_data_len = math.ceil(len(dataset) * 0.9087)
# train_data = scaled_data[0:training_data_len, :]
# #print(data[:train_data.shape[0]].tail())
# 
# # create the scaled test data set
# test_data = scaled_data[training_data_len-5: , :]
# 
# # use 5 consecutive trading days as the unit step size sliding through the stock price data
# x_train_5, y_train_5, x_test_5, y_test_5 = reshape_data(train_data, test_data, 5)
# print("when sequence length is 5, data shape:", x_train_5.shape, y_train_5.shape, x_test_5.shape, y_test_5.shape)


login success!
logout success!
                  date       open       high        low      close
date                                                              
2022-01-04  2022-01-04  3649.1510  3651.8924  3610.0922  3632.3289
2022-01-05  2022-01-05  3628.2634  3628.2634  3583.4668  3595.1761
2022-01-06  2022-01-06  3581.2184  3594.4890  3559.8803  3586.0792
2022-01-07  2022-01-07  3588.9851  3607.2316  3577.0976  3579.5427
2022-01-10  2022-01-10  3572.7434  3593.5187  3555.1255  3593.5187
...                ...        ...        ...        ...        ...
2023-12-25  2023-12-25  2909.8751  2920.6323  2903.9870  2918.8126
2023-12-26  2023-12-26  2917.9708  2917.9708  2892.8373  2898.8787
2023-12-27  2023-12-27  2900.1491  2917.9564  2890.1650  2914.6138
2023-12-28  2023-12-28  2913.1096  2961.6476  2909.3886  2954.7035
2023-12-29  2023-12-29  2950.8372  2975.0319  2950.0173  2974.9348

[484 rows x 5 columns]
